In [3]:
!pip install scrapy

  Obtaining dependency information for scrapy from https://files.pythonhosted.org/packages/0d/4e/6c06a4cfbb9a1e243c975da6a09982f085b1a14939fb195ea9e3e2f2b206/Scrapy-2.11.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for Twisted>=18.9.0 from https://files.pythonhosted.org/packages/49/d2/7b3e869b983fbf29d770fc2893f8df7c1739c6ff03a2b926b4fc43e4263e/twisted-24.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for cssselect>=0.9.1 from https://files.pythonhosted.org/packages/06/a9/2da08717a6862c48f1d61ef957a7bba171e7eefa6c0aa0ceb96a140c2a6b/cssselect-1.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for itemloaders>=1.0.1 from https://files.pythonhosted.org/packages/6f/3d/5fef602e1f89dd2d48551609432809fcf791a63b304177b8193165c04b23/itemloaders-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for parsel>=1.5.0 from https://files.pythonhosted.org/packages/85/7e/e3f1a7ff69303a4e08a8742a285406e5786650d8218ff194743eff292a


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: C:\Users\57310\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: C:\Users\57310\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Obtaining dependency information for scrapy from https://files.pythonhosted.org/packages/0d/4e/6c06a4cfbb9a1e243c975da6a09982f085b1a14939fb195ea9e3e2f2b206/Scrapy-2.11.2-py2.py3-none-any.whl.metadata
  Using cached Scrapy-2.11.2-py2.py3-none-any.whl.metadata (5.3 kB)
  Obtaining dependency information for itemloaders>=1.0.1 from https://files.pythonhosted.org/packages/6f/3d/5fef602e1f89dd2d48551609432809fcf791a63b304177b8193165c04b23/itemloaders-1.3.1-py3-none-any.whl.metadata
  Using cached itemloaders-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Obtaining dependency information for pyOpenSSL>=21.0.0 from https://files.pythonhosted.org/packages/d9/dd/e0aa7ebef5168c75b772eda64978c597a9129b46be17779054652a7999e4/pyOpenSSL-24.2.1-py3-none-any.whl.metadata
  Using cached pyOpenSSL-24.2.1-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for service-identity>=18.1.0 from https://files.pythonhosted.org/packages/3b/92/44669afe6354a7bed9968013862118c401690d8b5a805bab75ac17

In [4]:
import os
import requests
import pandas as pd
import numpy as np

from scrapy import Selector
from datetime import datetime

import zipfile

## GDELT 1.0 GKG Data

## GDELT 1.0 Events Data

In [5]:
events_url = "http://data.gdeltproject.org/events/index.html"
response = requests.get(events_url)
sel = Selector(text = response.text)

start_date = datetime(2023, 8, 13)
end_date = datetime(2024, 8, 13)

# Get the links as a url that can be downloaded later
links = sel.xpath('//a/@href').extract()

downloadable_links = []
base = "http://data.gdeltproject.org/events/"
for link in links:
    if link.endswith('.zip'):
        date_str = link.split('.')[0]
        try:
            file_date = datetime.strptime(date_str, "%Y%m%d")
            if start_date <= file_date <= end_date:
                downloadable_links.append(base + link)
        except ValueError:
            continue

In [7]:
destination = "GDELT Event Files"

os.makedirs(destination, exist_ok=True)

for link in downloadable_links:
    file_name = link.split('/')[-1]
    file_path = os.path.join(destination, file_name)

    response = requests.get(link)

    with open(file_path, 'wb') as file:
        file.write(response.content)

    # print(f"Downloaded {file_name} to {file_path}")

    # Decompress the zip file
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(destination)
        # print(f"Extracted {file_name} to {destination}")

    # Delete the zip file after extraction
    os.remove(file_path)

print("All files downloaded and extracted.")

All files downloaded and extracted.


In [8]:
import pandas as pd

In [13]:
trial = pd.read_csv(r"GDELT Event Files/20230813.export.CSV", sep= "\t", header = None)

In [27]:
csv_columns = ['global_id',
 'day', # Date the event took place in YYYYMMDD format
 'month_year', # Alternative formating YYYYMM
 'year', # Year
 'fraction_date', # Alternative formating YYYY.FFFF, where FFFF is the percentage of the year completed by that day
# actor 1
 'actor1_code',
 'actor1_name', # Name of Actor 1
 'actor1_country_code',
 'actor1_known_group_code', # Which group the actor belongs to NGO/ IGO/ rebel group. Ex: United Nations
 'actor1_ethnic_code',
 'actor1_religion1_code',
 'actor1_religion2_code',
 'actor1_type1_code', # Type codes talk about roles, for example police forces
 'actor1_type2_code', # goverment, military, education, elites, media, etc
 'actor1_type3_code', # -
# actor 2
 'actor2_code',
 'actor2_name', # Name of actor 2
 'actor2_country_code',
 'actor2_known_group_code',
 'actor2_ethnic_code',
 'actor2_religion1_code',
 'actor2_religion2_code',
 'actor2_type1_code', # Same as in actor 1
 'actor2_type2_code', # -
 'actor2_type3_code', # -
# ----------------
 'is_root_event', # Binary. Says if it is the root event. Can give insight into importance
 'event_code',
 'event_base_code',
 'event_root_code',
 'quad_class', # Event taxonomy: 1. Verbal cooperation, 2. Material Cooperation, 3. Verbal Conflict, 4. Material Conflict
 'goldstein_scale', # Numeric score from -10 to +10 capturing potential impact that the event will have in countries stability
 'num_mentions', # Number of mentions of the event across all documents. Can be seen as importance measure
 'num_sources', # Number of information sources containing mentions of the event
 'num_articles',# Number of source documents containing mentions of this event
 'avg_tone', # Avg tone of documents that mention the event. Goes from -100 (extremely negative) to 100 (extremely positive)
# actor 1 geo
 'actor1_geo_type', # Maps to: 1.Country, 2. US State, 3. US City, 4. World city, 5. World State
 'actor1_geo_full_name', # Name of location
 'actor1_geo_country_code',
 'actor1_geo_adm1_code',
 'actor1_geo_lat', # Latitude
 'actor1_geo_long', # Longitude
 'actor1_geo_feature_id',
# actor 2 geo
 'actor2_geo_type', # Check actor 1
 'actor2_geo_fullname',
 'actor2_geo_countrycode',
 'actor2_geo_adm1_code',
 'actor2_geo_lat',
 'actor2_geo_long',
 'actor2_geo_feature_id',
# action geo
 'action_geo_type', # Check actor 1
 'action2_geo_full_name',
 'action_geo_country_code',
 'action_geo_adm1_code',
 'action_geo_lat',
 'action_geo_long',
 'action_geo_feature_id',
# date and url
 'date_added', # Date the event was added to master database
 'source_url'] # URL

In [28]:
trial.columns = csv_columns

In [29]:
for i in range(1,10):
    df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)
    df.columns = csv_columns
    print(df.shape)

C:\Users\57310\AppData\Local\Temp/ipykernel_7416/1361513654.py:2: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)


(148860, 58)
(163824, 58)


C:\Users\57310\AppData\Local\Temp/ipykernel_7416/1361513654.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)


(165355, 58)


C:\Users\57310\AppData\Local\Temp/ipykernel_7416/1361513654.py:2: DtypeWarning: Columns (11,14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)


(164608, 58)
(158187, 58)
(102062, 58)
(85853, 58)


C:\Users\57310\AppData\Local\Temp/ipykernel_7416/1361513654.py:2: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)


(149442, 58)


C:\Users\57310\AppData\Local\Temp/ipykernel_7416/1361513654.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"GDELT Event Files/2023091{i}.export.CSV", sep= "\t", header = None)


(167824, 58)


In [22]:
df.iloc[:,24].value_counts()

is_root_event
MIL    53
GOV    26
BUS    11
MED    11
ELI     6
AGR     5
LEG     4
COP     2
LAB     2
CVL     2
ENV     1
Name: count, dtype: int64

In [32]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/b0/68/cf905fd2db4a84dc9b46803512b9765a3e9a6dfaa378a67c8db910c44ab3/streamlit-1.37.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for rich<14,>=10.14.0 from https://files.pythonhosted.org/packages/87/67/a37f6214d0e9fe57f6ae54b2956d550ca8365857f42a1ce0392bb21d9410/rich-13.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for toml<2,>=0.10.1 from https://files.pythonhosted.org/packages/44/6f/7120676b6d73228c96e17f1f794d8ab046fc910d781c8d151120c3f1569e/toml-0.10.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc457c723931fed3f/GitPython-3.1.43-py3-none-any.whl.metadata
  Obtaining dependency information for watchdog<5,>=2.1.5 from https://files.pythonhosted.org/packages/99/2e/b69dfaae7a83ea64ce36538cc103a3065e12c4479637


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: C:\Users\57310\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
from newspaper import Article
import spacy

# Initialize spacy NER model
    # nlp = spacy.load("en_core_web_sm")

# List of URLs
# urls = [
#     "https://theafronews.com/price-fixing-accusations-cast-shadow-on-food-industry-giants/",
#     "https://theafronews.com/price-fixing-accusations-cast-shadow-on-food-industry-giants/",
#     "https://www.wkrb13.com/2023/08/12/ingalls-snyder-llc-sells-8972-shares-of-bank-of-america-co-nysebac.html",
#     "https://www.thetimes.co.uk/article/questions-over-who-really-owns-ppe-firm-linked-to-mone-given-200m-0t3vk5pbl"
# ]

# Function to extract content from a URL
def extract_article_text(url):
    article = Article(url)
    article.download()
    try:
        article.parse()
    except:
        pass
    return article.title

# Function to perform NER on the article title
    def perform_ner(text):
        doc = nlp(text)
        return [(ent.text, ent.label_) for ent in doc.ents]

# Process each URL
# for url in urls:
#     title, text = extract_article_text(url)
#     entities = perform_ner(title)
#     print(f"Title: {title}")
#     print(f"Entities: {entities}")
#     print("-" * 40)


In [31]:
prueba = df[:100]
prueba['titulo'] = prueba['source_url'].apply(extract_article_text)
prueba

C:\Users\57310\AppData\Local\Temp/ipykernel_7416/3281536321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba['titulo'] = prueba['source_url'].apply(extract_article_text)


,global_id,day,month_year,year,fraction_date,actor1_code,actor1_name,actor1_country_code,actor1_known_group_code,actor1_ethnic_code,...,action_geo_type,action2_geo_full_name,action_geo_country_code,action_geo_adm1_code,action_geo_lat,action_geo_long,action_geo_feature_id,date_added,source_url,titulo
0,1127698478,20220919,202209,2022,2022.7096,NaN,NaN,NaN,NaN,NaN,...,4,"Melbourne, Victoria, Australia",AS,AS07,-37.8167,144.967000,-1586844,20230919,https://thedriven.io/2023/09/19/tesla-produces...,"Tesla produces its five millionth EV, takes ju..."
1,1127698479,20220919,202209,2022,2022.7096,NaN,NaN,NaN,NaN,NaN,...,2,"Ohio, United States",US,USOH,40.3736,-82.775500,OH,20230919,https://www.sbsun.com/2023/09/18/early-ontario...,Early Ontario businesswoman saw promise in the...
2,1127698480,20220919,202209,2022,2022.7096,AUS,AUSTRALIA,AUS,NaN,NaN,...,4,"Melbourne, Victoria, Australia",AS,AS07,-37.8167,144.967000,-1586844,20230919,https://thedriven.io/2023/09/19/tesla-produces...,"Tesla produces its five millionth EV, takes ju..."
3,1127698481,20220919,202209,2022,2022.7096,EDU,SCHOOL,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,20230919,https://www.cbs42.com/news/local/governor-ivey...,
4,1127698482,20220919,202209,2022,2022.7096,GBR,BRITISH,GBR,NaN,NaN,...,4,"London, London, City of, United Kingdom",UK,UKH9,51.5000,-0.116667,-2601889,20230919,https://www.theartnewspaper.com/2023/09/18/new...,New plaques on controversial City of London sc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1127698573,20230919,202309,2023,2023.7096,NaN,NaN,NaN,NaN,NaN,...,5,"Northern Territory, Northern Territory, Australia",AS,AS03,-20.0000,134.000000,-1592100,20230919,https://www.australiandefence.com.au/news/news...,Boosting ADF's maritime patrol and response ai...
96,1127698574,20230919,202309,2023,2023.7096,NaN,NaN,NaN,NaN,NaN,...,4,"Yarra, New South Wales, Australia",AS,AS02,-34.7833,149.633000,-1611642,20230919,https://www.dailymail.co.uk/tvshowbiz/article-...,Australian swimming champion and Olympic gold ...
97,1127698575,20230919,202309,2023,2023.7096,NaN,NaN,NaN,NaN,NaN,...,4,"Toowoomba, Queensland, Australia",AS,AS04,-27.5606,151.954000,-1605321,20230919,https://www.couriermail.com.au/lifestyle/food/...,
98,1127698576,20230919,202309,2023,2023.7096,NaN,NaN,NaN,NaN,NaN,...,1,Australia,AS,AS,-25.0000,135.000000,AS,20230919,https://www.thecourier.com.au/story/8354777/au...,Australia struggles to hold cyber criminals to...


In [33]:
import pandas as pd
df = pd.read_csv(r"masterData.csv")

C:\Users\57310\AppData\Local\Temp/ipykernel_7416/2555524143.py:2: DtypeWarning: Columns (24,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"masterData.csv")


In [34]:
df

,global_id,day,month_year,year,fraction_date,actor1_code,actor1_name,actor1_country_code,actor1_known_group_code,actor1_ethnic_code,...,actor2_geo_feature_id,action_geo_type,action2_geo_full_name,action_geo_country_code,action_geo_adm1_code,action_geo_lat,action_geo_long,action_geo_feature_id,date_added,source_url
0,1120554548,20220813,202208,2022,2022.6110,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20230813,https://www.bostonglobe.com/2023/08/12/metro/m...
1,1120554549,20220813,202208,2022,2022.6110,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20230813,https://www.thetimes.co.uk/article/tougher-a-l...
2,1120554551,20220813,202208,2022,2022.6110,NaN,NaN,NaN,NaN,NaN,...,295004,3,"Miami, Florida, United States",US,USFL,25.7743,-80.1937,295004,20230813,https://www.kmbc.com/article/amazon-founder-je...
3,1120554553,20220813,202208,2022,2022.6110,NaN,NaN,NaN,NaN,NaN,...,465961,3,"Des Moines, Iowa, United States",US,USIA,41.6005,-93.6091,465961,20230813,https://gazette.com/news/us-world/trump-swoops...
4,1120554556,20220813,202208,2022,2022.6110,CHN,CHINA,CHN,NaN,NaN,...,CH,1,Malaysia,MY,MY,2.5000,112.5000,MY,20230813,https://www.thestar.com.my/news/nation/2023/08...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2823899,1141475878,20231120,202311,2023,2023.8767,USAGOVBUS,THE US,USA,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20231120,https://biztoc.com/x/fe43f06c38b575f5
2823900,1141475892,20231120,202311,2023,2023.8767,USALAB,FLORIDA,USA,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20231120,https://navarrepress.com/ex-executives-lawsuit...
2823901,1141475909,20231120,202311,2023,2023.8767,ZWE,ZIMBABWE,ZWE,NaN,NaN,...,NaN,4,"Chiwenga, Masvingo, Zimbabwe",ZI,ZI08,-20.1148,31.0356,10676003,20231120,https://www.thezimbabwemail.com/economic-analy...
2823902,1141475910,20231120,202311,2023,2023.8767,mao,MAORI,NaN,NaN,mao,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20231120,https://waateanews.com/2023/11/21/te-reo-purge...
